In [1]:
import sqlite3 as sql
sql.headers = 'on'
sql.mode = 'column'

In [2]:
conn = sql.connect('sqlite_db_pythonsqlite.db')

In [3]:
cursor = conn.cursor()

In [4]:
query1 = """
    SELECT *
    FROM FACILITIES
    """
cursor.execute(query1)
rows = cursor.fetchall()
 
for row in rows:
    print(row)

(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [5]:
with open('SQLTasks Tier 2.sql', 'r') as file:
    lines = file.readlines()
    lines = [line.replace('\n', '') for line in lines]
    lines = [line.replace(',', '') for line in lines]
    lines = [line.replace('/*', '') for line in lines]
    lines = [line.replace('*/', '') for line in lines]
for line in lines:
    print(line)

 Welcome to the SQL mini project. You will carry out this project partly in
the PHPMyAdmin interface and partly in Jupyter via a Python connection.

This is Tier 2 of the case study which means that there'll be less guidance for you about how to setup
your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
you might need to do some digging aand revise the Working with Relational Databases in Python chapter in the previous resource.

Otherwise the questions in the case study are exactly the same as with Tier 1. 

PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser and
using the following Username and Password:

URL: https://sql.springboard.com/
Username: student
Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:
    i) the "Bookings" table
    ii) the "Facilities" table and


In [6]:
query = """SELECT * 
FROM facilities"""
rs = cursor.execute(query).fetchall()
colnames = cursor.description
for row in colnames:
    print(row[0])

facid
name
membercost
guestcost
initialoutlay
monthlymaintenance


In [7]:
query = 'SELECT * FROM members'
rs = cursor.execute(query).fetchall()
colnames = cursor.description
for row in colnames:
    print(row[0])

memid
surname
firstname
address
zipcode
telephone
recommendedby
joindate


In [8]:
query = 'SELECT * FROM bookings'
rs = cursor.execute(query).fetchall()
colnames = cursor.description
for row in colnames:
    print(row[0])

bookid
facid
memid
starttime
slots


In [9]:
def return_query(t_query):
    return cursor.execute(t_query).fetchall()

Q1: Some of the facilities charge a fee to members, but some do not.',
 'Write a SQL query to produce a list of the names of the facilities that do.

In [10]:
query = """SELECT name 
FROM facilities 
WHERE membercost != 0"""
return_query(query)

[('Tennis Court 1',),
 ('Tennis Court 2',),
 ('Massage Room 1',),
 ('Massage Room 2',),
 ('Squash Court',)]

Q2: How many facilities do not charge a fee to members?

In [11]:
query = 'SELECT COUNT(*) FROM facilities WHERE membercost == 0'
return_query(query)

[(4,)]

Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [12]:
query = """SELECT facid, name, membercost, monthlymaintenance 
FROM facilities 
WHERE membercost != 0 AND membercost < monthlymaintenance*.2"""
return_query(query)

[(0, 'Tennis Court 1', 5, 200),
 (1, 'Tennis Court 2', 5, 200),
 (4, 'Massage Room 1', 9.9, 3000),
 (5, 'Massage Room 2', 9.9, 3000),
 (6, 'Squash Court', 3.5, 80)]

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator. 

In [13]:
query = """SELECT * 
FROM facilities 
WHERE facid IN (1, 5)"""
return_query(query)

[(1, 'Tennis Court 2', 5, 25, 8000, 200),
 (5, 'Massage Room 2', 9.9, 80, 4000, 3000)]

Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [14]:
q = '''SELECT name, monthlymaintenance,
CASE WHEN (monthlymaintenance > 100) 
THEN 'expensive' 
ELSE 'cheap' 
END as cost
FROM facilities'''
return_query(q)

[('Tennis Court 1', 200, 'expensive'),
 ('Tennis Court 2', 200, 'expensive'),
 ('Badminton Court', 50, 'cheap'),
 ('Table Tennis', 10, 'cheap'),
 ('Massage Room 1', 3000, 'expensive'),
 ('Massage Room 2', 3000, 'expensive'),
 ('Squash Court', 80, 'cheap'),
 ('Snooker Table', 15, 'cheap'),
 ('Pool Table', 15, 'cheap')]

Q6: You'd like to get the first and last name of the last member(s), who signed up. Try not to use the LIMIT clause for your solution.

In [15]:
query= '''SELECT surname, firstname 
FROM members 
WHERE rowid = (SELECT MAX(rowid) FROM members) 
ORDER BY joindate'''
return_query(query)

[('Smith', 'Darren')]

Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [16]:
query = """SELECT mem.firstname || ' ' || mem.surname, fac.name
FROM members mem, facilities fac 
WHERE
memid IN (SELECT DISTINCT memid FROM bookings WHERE facid IN (0,1)) 
AND memid != 0 
AND facid IN (0,1)
ORDER BY surname, firstname"""
return_query(query)

[('Florence Bader', 'Tennis Court 1'),
 ('Florence Bader', 'Tennis Court 2'),
 ('Anne Baker', 'Tennis Court 1'),
 ('Anne Baker', 'Tennis Court 2'),
 ('Timothy Baker', 'Tennis Court 1'),
 ('Timothy Baker', 'Tennis Court 2'),
 ('Tim Boothe', 'Tennis Court 1'),
 ('Tim Boothe', 'Tennis Court 2'),
 ('Gerald Butters', 'Tennis Court 1'),
 ('Gerald Butters', 'Tennis Court 2'),
 ('Joan Coplin', 'Tennis Court 1'),
 ('Joan Coplin', 'Tennis Court 2'),
 ('Erica Crumpet', 'Tennis Court 1'),
 ('Erica Crumpet', 'Tennis Court 2'),
 ('Nancy Dare', 'Tennis Court 1'),
 ('Nancy Dare', 'Tennis Court 2'),
 ('David Farrell', 'Tennis Court 1'),
 ('David Farrell', 'Tennis Court 2'),
 ('Jemima Farrell', 'Tennis Court 1'),
 ('Jemima Farrell', 'Tennis Court 2'),
 ('Matthew Genting', 'Tennis Court 1'),
 ('Matthew Genting', 'Tennis Court 2'),
 ('John Hunt', 'Tennis Court 1'),
 ('John Hunt', 'Tennis Court 2'),
 ('David Jones', 'Tennis Court 1'),
 ('David Jones', 'Tennis Court 2'),
 ('Douglas Jones', 'Tennis Court 1')

Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries. 

In [17]:
q = '''SELECT f.name, m.firstname ||' '||m.surname,
CASE WHEN b.memid = 0 
THEN b.slots*f.guestcost
ELSE b.slots*f.membercost
END AS cost
FROM bookings b, facilities f, members m
WHERE 
b.starttime LIKE "%2012-09-14%" AND
((b.memid = 0 and b.slots*f.guestcost > 30) OR
(b.memid != 0 and b.slots*f.membercost > 30)) AND
m.memid = b.memid
ORDER BY cost DESC
'''
return_query(q)

[('Massage Room 1', 'GUEST GUEST', 480),
 ('Massage Room 2', 'GUEST GUEST', 480),
 ('Massage Room 1', 'GUEST GUEST', 320),
 ('Massage Room 2', 'GUEST GUEST', 320),
 ('Massage Room 1', 'GUEST GUEST', 320),
 ('Massage Room 2', 'GUEST GUEST', 320),
 ('Massage Room 1', 'GUEST GUEST', 240),
 ('Massage Room 2', 'GUEST GUEST', 240),
 ('Massage Room 1', 'GUEST GUEST', 240),
 ('Massage Room 2', 'GUEST GUEST', 240),
 ('Massage Room 1', 'GUEST GUEST', 240),
 ('Massage Room 2', 'GUEST GUEST', 240),
 ('Massage Room 1', 'GUEST GUEST', 160),
 ('Massage Room 2', 'GUEST GUEST', 160),
 ('Massage Room 1', 'GUEST GUEST', 160),
 ('Massage Room 2', 'GUEST GUEST', 160),
 ('Massage Room 1', 'GUEST GUEST', 160),
 ('Massage Room 2', 'GUEST GUEST', 160),
 ('Massage Room 1', 'GUEST GUEST', 160),
 ('Massage Room 2', 'GUEST GUEST', 160),
 ('Massage Room 1', 'GUEST GUEST', 160),
 ('Massage Room 2', 'GUEST GUEST', 160),
 ('Tennis Court 1', 'GUEST GUEST', 150),
 ('Tennis Court 2', 'GUEST GUEST', 150),
 ('Squash Court'

Q9: This time, produce the same result as in Q8, but using a subquery.

In [18]:
q = '''
SELECT facname, memberid, m.firstname ||' '||m.surname, cost
FROM
(SELECT f.name AS facname, b.memid AS memberid,
CASE WHEN b.memid = 0 
THEN b.slots*f.guestcost
ELSE b.slots*f.membercost
END AS cost
FROM bookings b, facilities f
WHERE
b.starttime LIKE "%2012-09-14%" AND
((b.memid = 0 and b.slots*f.guestcost > 30) OR
(b.memid != 0 and b.slots*f.membercost > 30))) AS derivedTable
INNER JOIN members m
ON memberid = m.memid
ORDER BY cost DESC
'''
return_query(q)

[('Massage Room 1', 0, 'GUEST GUEST', 480),
 ('Massage Room 2', 0, 'GUEST GUEST', 480),
 ('Massage Room 1', 0, 'GUEST GUEST', 320),
 ('Massage Room 2', 0, 'GUEST GUEST', 320),
 ('Massage Room 1', 0, 'GUEST GUEST', 320),
 ('Massage Room 2', 0, 'GUEST GUEST', 320),
 ('Massage Room 1', 0, 'GUEST GUEST', 240),
 ('Massage Room 2', 0, 'GUEST GUEST', 240),
 ('Massage Room 1', 0, 'GUEST GUEST', 240),
 ('Massage Room 2', 0, 'GUEST GUEST', 240),
 ('Massage Room 1', 0, 'GUEST GUEST', 240),
 ('Massage Room 2', 0, 'GUEST GUEST', 240),
 ('Massage Room 1', 0, 'GUEST GUEST', 160),
 ('Massage Room 2', 0, 'GUEST GUEST', 160),
 ('Massage Room 1', 0, 'GUEST GUEST', 160),
 ('Massage Room 2', 0, 'GUEST GUEST', 160),
 ('Massage Room 1', 0, 'GUEST GUEST', 160),
 ('Massage Room 2', 0, 'GUEST GUEST', 160),
 ('Massage Room 1', 0, 'GUEST GUEST', 160),
 ('Massage Room 2', 0, 'GUEST GUEST', 160),
 ('Massage Room 1', 0, 'GUEST GUEST', 160),
 ('Massage Room 2', 0, 'GUEST GUEST', 160),
 ('Tennis Court 1', 0, 'GUEST GU

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [19]:
query = '''SELECT mems.surname, mems.firstname, recs.surname AS rec_last_name, recs.firstname AS rec_first_name
FROM members mems
INNER JOIN members recs
ON recs.memid = mems.recommendedby
ORDER BY mems.surname, mems.firstname'''
return_query(query)

[('Bader', 'Florence', 'Stibbons', 'Ponder'),
 ('Baker', 'Anne', 'Stibbons', 'Ponder'),
 ('Baker', 'Timothy', 'Farrell', 'Jemima'),
 ('Boothe', 'Tim', 'Rownam', 'Tim'),
 ('Butters', 'Gerald', 'Smith', 'Darren'),
 ('Coplin', 'Joan', 'Baker', 'Timothy'),
 ('Crumpet', 'Erica', 'Smith', 'Tracy'),
 ('Dare', 'Nancy', 'Joplette', 'Janice'),
 ('Genting', 'Matthew', 'Butters', 'Gerald'),
 ('Hunt', 'John', 'Purview', 'Millicent'),
 ('Jones', 'David', 'Joplette', 'Janice'),
 ('Jones', 'Douglas', 'Jones', 'David'),
 ('Joplette', 'Janice', 'Smith', 'Darren'),
 ('Mackenzie', 'Anna', 'Smith', 'Darren'),
 ('Owen', 'Charles', 'Smith', 'Darren'),
 ('Pinker', 'David', 'Farrell', 'Jemima'),
 ('Purview', 'Millicent', 'Smith', 'Tracy'),
 ('Rumney', 'Henrietta', 'Genting', 'Matthew'),
 ('Sarwin', 'Ramnaresh', 'Bader', 'Florence'),
 ('Smith', 'Jack', 'Smith', 'Darren'),
 ('Stibbons', 'Ponder', 'Tracy', 'Burton'),
 ('Worthington-Smyth', 'Henry', 'Smith', 'Tracy')]

Q12: Find the facilities with their usage by member, but not guests 

In [20]:
q = '''SELECT f.name, SUM(b.slots/2.0) AS hours
FROM facilities AS f
INNER JOIN bookings AS b
USING (facid)
WHERE b.memid != 0
GROUP BY f.facid'''
return_query(q)

[('Tennis Court 1', 478.5),
 ('Tennis Court 2', 441.0),
 ('Badminton Court', 543.0),
 ('Table Tennis', 397.0),
 ('Massage Room 1', 442.0),
 ('Massage Room 2', 27.0),
 ('Squash Court', 209.0),
 ('Snooker Table', 430.0),
 ('Pool Table', 428.0)]

Q13: Find the facilities usage by month, but not guests 

In [22]:
q = '''SELECT f.name, strftime('%m', b.starttime) AS month, SUM(b.slots)/2 AS hours
FROM facilities AS f
INNER JOIN bookings b
ON f.facid = b.facid
WHERE b.memid != 0
GROUP BY month, f.name
'''
return_query(q)

[('Badminton Court', '07', 82),
 ('Massage Room 1', '07', 83),
 ('Massage Room 2', '07', 4),
 ('Pool Table', '07', 55),
 ('Snooker Table', '07', 70),
 ('Squash Court', '07', 25),
 ('Table Tennis', '07', 49),
 ('Tennis Court 1', '07', 100),
 ('Tennis Court 2', '07', 61),
 ('Badminton Court', '08', 207),
 ('Massage Room 1', '08', 158),
 ('Massage Room 2', '08', 9),
 ('Pool Table', '08', 151),
 ('Snooker Table', '08', 158),
 ('Squash Court', '08', 92),
 ('Table Tennis', '08', 148),
 ('Tennis Court 1', '08', 169),
 ('Tennis Court 2', '08', 172),
 ('Badminton Court', '09', 253),
 ('Massage Room 1', '09', 201),
 ('Massage Room 2', '09', 14),
 ('Pool Table', '09', 221),
 ('Snooker Table', '09', 202),
 ('Squash Court', '09', 92),
 ('Table Tennis', '09', 200),
 ('Tennis Court 1', '09', 208),
 ('Tennis Court 2', '09', 207)]